In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob, os
import csv
from datetime import datetime
import copy

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
import tqdm         # make loops show as a smart progress meter

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import os

In [27]:
# 1) Classic_Model (ReLu,  bias, not orthogonal, Softmax)========================================================================================================
class ClassicalModel_Relu_Bias(nn.Module):
    def __init__(self, hidden_sizes, input_size=4, output_size=3):
        """
        input_size: number of input features (e.g. 4 for Iris)
        hidden_sizes: list of hidden layer sizes, e.g. [8, 8, 4]
        output_size: number of output classes (e.g. 3 for Iris)
        """
        super(ClassicalModel_Relu_Bias, self).__init__()
        
        # Create a list of layer sizes
        layer_sizes = [input_size] + hidden_sizes + [output_size]
        
        # Create a list of Linear layers
        self.layers = nn.ModuleList([
            nn.Linear(layer_sizes[i], layer_sizes[i+1])
            for i in range(len(layer_sizes) - 1)
        ])

    def forward(self, x):
        # Go through all layers except the last one with ReLU
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        # Apply softmax on the last layer (output)
        x = F.softmax(self.layers[-1](x), dim=1)
        return x

In [28]:
# 2) Classic_Model_NoBias (ReLU, no bias, Not orthogonal, SoftMax)============================================================================================
class ClassicalModel_ReLU_NoBias(nn.Module):
    def __init__(self, hidden_sizes,input_size=4, output_size=3):
        """
        ReLU, no bias, standard weights, SoftMax
        """
        super(ClassicalModel_ReLU_NoBias, self).__init__()
        layer_sizes = [input_size] + hidden_sizes + [output_size]

        self.layers = nn.ModuleList([
            nn.Linear(layer_sizes[i], layer_sizes[i+1], bias=False)
            for i in range(len(layer_sizes)-1)
        ])

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        x = F.softmax(self.layers[-1](x), dim=1)
        return x

In [29]:
# 3) Classic_Model_NoRelu (No ReLu, bias, Not orthogonal, SoftMax)========================================================================================================
class ClassicalModel_NoRelu_Bias(nn.Module):
    def __init__(self, hidden_sizes, input_size=4, output_size=3):
        """
        input_size: number of input features (e.g. 4 for Iris)
        hidden_sizes: list of hidden layer sizes, e.g. [8, 8, 4]
        output_size: number of output classes (e.g. 3 for Iris)
        """
        super(ClassicalModel_NoRelu_Bias, self).__init__()
        
        # Create a list of layer sizes
        layer_sizes = [input_size] + hidden_sizes + [output_size]
        
        # Create a list of Linear layers
        self.layers = nn.ModuleList([
            nn.Linear(layer_sizes[i], layer_sizes[i+1], bias=False)
            for i in range(len(layer_sizes)-1)
        ])

    def forward(self, x): #defines how an input x becomes an output, passing through the network’s layers.
        # Go through all layers except the last one with ReLU
        for layer in self.layers[:-1]:
            x = layer(x)
        # Apply softmax on the last layer (output)
        x = F.softmax(self.layers[-1](x), dim=1)
        return x

In [30]:
# 4) Classic_Model_NoReLU_NoBias (No ReLU, no bias, Not orthogonal, SoftMax)===============================================================================
class ClassicalModel_NoReLU_NoBias(nn.Module):
    def __init__(self, hidden_sizes, input_size=4, output_size=3):
        """
        input_size: number of input features (e.g. 4 for Iris)
        hidden_sizes: list of hidden layer sizes, e.g. [8, 8, 4]
        output_size: number of output classes (e.g. 3 for Iris)
        """
        super(ClassicalModel_NoReLU_NoBias, self).__init__()
        layer_sizes = [input_size] + hidden_sizes + [output_size]

        self.layers = nn.ModuleList([
            nn.Linear(layer_sizes[i], layer_sizes[i+1], bias=False)
            for i in range(len(layer_sizes)-1)
        ])

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = layer(x)
        x = F.softmax(self.layers[-1](x), dim=1)
        return x

In [31]:
# 5) Classic_Model_Ortho_ReLU_Bias (ReLU, bias, orthogonal, SoftMax)======================================================================================
class ClassicalModel_Ortho_ReLU_Bias(nn.Module):
    def __init__(self, hidden_sizes,input_size=4, output_size=3):
        """
        ReLU, bias, orthogonal weights, SoftMax
        """
        super(ClassicalModel_Ortho_ReLU_Bias, self).__init__()
        layer_sizes = [input_size] + hidden_sizes + [output_size]

        self.layers = nn.ModuleList([
            nn.utils.parametrizations.orthogonal(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            for i in range(len(layer_sizes)-1)
        ])

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        x = F.softmax(self.layers[-1](x), dim=1)
        return x

In [32]:
# 6) Classic_Model_Ortho_ReLU_NoBias (ReLU, no bias, orthogonal, SoftMax)================================================================================
class ClassicalModel_Ortho_ReLU_NoBias(nn.Module):
    def __init__(self, hidden_sizes, input_size=4, output_size=3):
        """
        ReLU, no bias, orthogonal weights, SoftMax
        """
        super(ClassicalModel_Ortho_ReLU_NoBias, self).__init__()
        layer_sizes = [input_size] + hidden_sizes + [output_size]

        self.layers = nn.ModuleList([
            nn.utils.parametrizations.orthogonal(nn.Linear(layer_sizes[i], layer_sizes[i+1], bias=False))
            for i in range(len(layer_sizes)-1)
        ])

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        x = F.softmax(self.layers[-1](x), dim=1)
        return x

In [33]:
# 7) Classic_Model_Ortho_NoReLU_Bias (No ReLU, bias, orthogonal, SoftMax)===============================================================================
class ClassicalModel_Ortho_NoReLU_Bias(nn.Module):
    def __init__(self, hidden_sizes, input_size=4, output_size=3):
        """
        No ReLU, bias, orthogonal weights, SoftMax
        """
        super(ClassicalModel_Ortho_NoReLU_Bias, self).__init__()
        layer_sizes = [input_size] + hidden_sizes + [output_size]

        self.layers = nn.ModuleList([
            nn.utils.parametrizations.orthogonal(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            for i in range(len(layer_sizes)-1)
        ])

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = layer(x)
        x = F.softmax(self.layers[-1](x), dim=1)
        return x

In [34]:

# 8) Classic_Model_Ortho_NoReLU_NoBias (No ReLU, no bias, orthogonal, SoftMax)=========================================================================
class ClassicalModel_Ortho_NoReLU_NoBias(nn.Module):
    def __init__(self, hidden_sizes,input_size=4, output_size=3):
        """
        No ReLU, no bias, orthogonal weights, SoftMax
        """
        super(ClassicalModel_Ortho_NoReLU_NoBias, self).__init__()
        layer_sizes = [input_size] + hidden_sizes + [output_size]

        self.layers = nn.ModuleList([
            nn.utils.parametrizations.orthogonal(nn.Linear(layer_sizes[i], layer_sizes[i+1], bias=False))
            for i in range(len(layer_sizes)-1)
        ])

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = layer(x)
        x = F.softmax(self.layers[-1](x), dim=1)
        return x

In [ ]:
class UnitaryLayer(nn.Module):
    """
    Complex square unitary layer using matrix exponential.
    Models an ideal MZI mesh.
    """

    def __init__(self, width):
        super().__init__()
        self.width = width

        # Trainable complex matrix (skew-Hermitian enforced in forward)
        self.A = nn.Parameter(torch.randn(width, width, dtype=torch.complex64) * 1e-3)

    def forward(self, E):
        """
        E: complex electric field, shape (batch, width)
        Returns:
            E_out: complex field after unitary transformation
        """
        # Enforce skew-Hermitian: A† = -A
        A_skew = self.A - self.A.conj().T

        # Exponentiate → exact unitary
        U = torch.matrix_exp(A_skew)  # shape (width, width)

        # Multiply input field by unitary
        # batch @ width times width → (batch, width)
        return E @ U.T

class PhotonicNNMatrixExp(nn.Module):
    def __init__(self, width, depth, input_size, output_size, nonlinearity="relu"):
        super().__init__()
        self.width = width
        self.depth = depth
        self.nonlinearity = nonlinearity
        self.meshes = nn.ModuleList([UnitaryLayer(width) for _ in range(depth)])
        self.classifier = nn.Linear(width, output_size, bias=True)

    # --- Methods for photonic operations ---
    def photodetect(self, E):
        return torch.real(E * E.conj())

    def eo_nonlinearity(self, I):
        if self.nonlinearity == "relu":
            return torch.relu(I)
        elif self.nonlinearity == "tanh":
            return torch.tanh(I)
        elif self.nonlinearity == "none":
            return I
        else:
            raise ValueError("Unknown nonlinearity")

    def intensity_to_field(self, I):
        return torch.sqrt(torch.clamp(I, min=1e-12)).to(torch.complex64)

    def forward(self, I_input):
        batch_size = I_input.shape[0]
        I = torch.zeros(batch_size, self.width, device=I_input.device)
        I[:, :I_input.shape[1]] = I_input

        E = self.intensity_to_field(I)

        for mesh in self.meshes:
            E = mesh(E)
            I = self.photodetect(E)
            I = self.eo_nonlinearity(I)
            E = self.intensity_to_field(I)

        I_out = self.photodetect(E)
        return torch.softmax(self.classifier(I_out), dim=1)

In [50]:
class train_backprop():

    def __init__(self) -> None:
        iris = load_iris()
        X = iris['data']
        y = torch.tensor(iris['target'], dtype=torch.long)

        scaler = StandardScaler()
        X_scaled = torch.tensor(scaler.fit_transform(X), dtype=torch.float)

        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y, test_size=0.2, random_state=2)  # Total n dat 150, so 120 for train and 30 for test

        self.train_data = TensorDataset(X_train, y_train)
        test_data = TensorDataset(X_test, y_test)

        self.train_loader = DataLoader(self.train_data, shuffle=True, batch_size=12)  # batch size 12 so 10 batch
        self.test_loader = DataLoader(test_data, batch_size=len(test_data.tensors[0]))  # batch size 30 max

    def train_IRIS(self, model, epochs: int = 400, epsilon: float = 0.02,
               log_first_layer=False, max_snapshots=10):

        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        loss_fn = nn.CrossEntropyLoss()

        train_loss = np.zeros(epochs)
        valid_loss = np.zeros(epochs)
        train_accuracy = np.zeros(epochs)
        valid_accuracy = np.zeros(epochs)

        # 🔹 storage for matrices
        first_layer_matrices = []

        snapshot_count = 0

        for epoch in tqdm.trange(epochs):
            for X, y in self.train_loader:
                y_pred = model(X)
                loss = loss_fn(y_pred, y)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # 🔹 log first-layer matrix
                if log_first_layer and snapshot_count < max_snapshots:
                    W = model.layers[0].weight.detach().cpu().numpy()
                    first_layer_matrices.append(W.copy())
                    snapshot_count += 1

            # --- metrics (unchanged) ---
            with torch.no_grad():
                X_train, y_train = self.train_data[:]
                y_pred_train = model(X_train)
                train_loss[epoch] = loss_fn(y_pred_train, y_train).item()
                y_pred_labels = torch.argmax(y_pred_train, axis=1)
                train_accuracy[epoch] = torch.mean((y_pred_labels == y_train).float()).item()

            with torch.no_grad():
                X_val, y_val = next(iter(self.test_loader))
                y_pred_val = model(X_val)
                valid_loss[epoch] = loss_fn(y_pred_val, y_val).item()
                y_pred_labels = torch.argmax(y_pred_val, axis=1)
                valid_accuracy[epoch] = torch.mean((y_pred_labels == y_val).float()).item()

        final_train_loss = train_loss[-1]
        final_valid_loss = valid_loss[-1]

        epochs_to_final_train = np.argmax(train_loss <= final_train_loss * (1 + epsilon)) + 1
        epochs_to_final_valid = np.argmax(valid_loss <= final_valid_loss * (1 + epsilon)) + 1

        return (train_loss, valid_loss, train_accuracy, valid_accuracy,
                epochs_to_final_train, epochs_to_final_valid,
                first_layer_matrices)

        

    def graph_train_IRIS(self, model, epochs: int = 400, n_runs: int = 15, epsilon: float = 0.02) -> None:
        """
        Train the given model multiple times (n_runs) and save all results in a single CSV file.
        For the FIRST run only, print the first 10 first-layer weight matrices
        (after optimizer steps) to verify orthogonality.
        """
        import os
        from datetime import datetime
        import pandas as pd
        import matplotlib.pyplot as plt

        os.makedirs("results", exist_ok=True)

        model_name = model.__class__.__name__

        # Detect hidden layer sizes
        try:
            hidden_sizes = [layer.out_features for layer in model.layers[:-1]]
        except AttributeError:
            hidden_sizes = ["Unknown"]

        global_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        all_runs = []

        for run in range(1, n_runs + 1):
            # ⚡ pass hidden_sizes explicitly
            model_instance = copy.deepcopy(model)
            print(f"\n🚀 Run {run}/{n_runs} for {model_name} with layers {hidden_sizes}")

            optimizer = torch.optim.Adam(model_instance.parameters(), lr=0.001)
            loss_fn = nn.CrossEntropyLoss()

            train_loss = np.zeros(epochs)
            valid_loss = np.zeros(epochs)
            train_accuracy = np.zeros(epochs)
            valid_accuracy = np.zeros(epochs)

            printed = 0  # number of matrices printed so far

            for epoch in range(epochs):
                # --- TRAIN ---
                for X, y in self.train_loader:
                    y_pred = model_instance(X)
                    loss = loss_fn(y_pred, y)

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    # 🔹 PRINT FIRST 10 MATRICES (only run 1)
                    if run == 1 and printed < 10:
                        try:
                            # Classical NN
                            W = model_instance.layers[0].weight.detach().cpu().numpy()
                        except AttributeError:
                            # Photonic NN
                            W = model_instance.meshes[0].A.detach().cpu().numpy()
                        print(f"\n--- First-layer weight matrix (step {printed + 1}) ---")
                        print(W)

                        # Orthogonality check
                        try:
                            # Classical
                            if W.shape[0] <= W.shape[1]:
                                M = W @ W.T
                                label = "W Wᵀ"
                            else:
                                M = W.T @ W
                                label = "Wᵀ W"
                        except Exception:
                            # Photonic: A is complex, check unitarity via matrix exponential
                            A_skew = model_instance.meshes[0].A - model_instance.meshes[0].A.conj().T
                            U = torch.matrix_exp(A_skew).detach().cpu().numpy()
                            M = U.conj().T @ U
                            label = "U† U (should be identity)"

                        print(f"{label} (should be close to identity):")
                        print(M)

                        printed += 1

                # --- TRAIN METRICS ---
                with torch.no_grad():
                    X_train, y_train = self.train_data[:]
                    y_pred_train = model_instance(X_train)
                    train_loss[epoch] = loss_fn(y_pred_train, y_train).item()
                    train_accuracy[epoch] = (
                        torch.argmax(y_pred_train, dim=1) == y_train
                    ).float().mean().item()

                # --- VALIDATION METRICS ---
                with torch.no_grad():
                    X_val, y_val = next(iter(self.test_loader))
                    y_pred_val = model_instance(X_val)
                    valid_loss[epoch] = loss_fn(y_pred_val, y_val).item()
                    valid_accuracy[epoch] = (
                        torch.argmax(y_pred_val, dim=1) == y_val
                    ).float().mean().item()

            # --- FINAL METRICS ---
            final_train_loss = train_loss[-1]
            final_valid_loss = valid_loss[-1]
            final_train_acc = train_accuracy[-1]
            final_valid_acc = valid_accuracy[-1]

            epochs_to_final_train = np.argmax(train_loss <= final_train_loss * (1 + epsilon)) + 1
            epochs_to_final_valid = np.argmax(valid_loss <= final_valid_loss * (1 + epsilon)) + 1

            all_runs.append({
                "timestamp": global_timestamp,
                "model_name": model_name,
                "hidden_sizes": str(hidden_sizes),
                "epochs": epochs,
                "run_number": run,
                "final_train_loss": final_train_loss,
                "final_valid_loss": final_valid_loss,
                "final_train_accuracy": final_train_acc,
                "final_valid_accuracy": final_valid_acc,
                "epochs_to_final_train": epochs_to_final_train,
                "epochs_to_final_valid": epochs_to_final_valid
            })

            print(f"🏁 Validation accuracy (run {run}): {final_valid_acc:.3f}")
            print(f"⏱️ Epochs to converge (train/valid): {epochs_to_final_train}/{epochs_to_final_valid}")

            # --- PLOT ONLY FOR LAST RUN ---
            if run == n_runs:
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

                ax1.plot(train_accuracy, label="Train accuracy")
                ax1.plot(valid_accuracy, label="Validation accuracy")
                ax1.set_title("Accuracy")
                ax1.set_xlabel("Epoch")
                ax1.set_ylim([0.0, 1.05])
                ax1.legend()

                ax2.plot(train_loss, label="Train loss")
                ax2.plot(valid_loss, label="Validation loss")
                ax2.set_title("Loss")
                ax2.set_xlabel("Epoch")
                ax2.legend()

                plt.tight_layout()
                plt.show()

        # --- SAVE RESULTS ---
        results_df = pd.DataFrame(all_runs)
        csv_filename = f"{model_name}_layers{hidden_sizes}_epochs{epochs}_{n_runs}runs_{global_timestamp}.csv"
        csv_path = os.path.join("results", csv_filename)
        results_df.to_csv(csv_path, index=False)

        print(f"\n✅ All {n_runs} runs saved in one file → {csv_path}")
    # ======================================================    



In [51]:
a = train_backprop()

for width in [4, 5]:
    for depth in [1, 2]:
        model = PhotonicNNMatrixExp(width=width,depth=depth, input_size=4, output_size=3, nonlinearity="relu")
        a.graph_train_IRIS(model)



🚀 Run 1/15 for PhotonicNNMatrixExp with layers ['Unknown']


TypeError: PhotonicNNMatrixExp.intensity_to_field() takes 1 positional argument but 2 were given